In [5]:
import sentencepiece as spm
import librosa
import numpy as np
from pydub import AudioSegment
import speech_recognition as sr
import os
import requests
import json
from pydub import AudioSegment

In [ ]:
## take audio and transcribe audio to text file
recognizer = sr.Recognizer()

# Load audio
audio_folder = "D:/coding/ai_assistant/training_data"
audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".mp3")]
transcription_folder = "D:/coding/ai_assistant/transcriptions" 

for audio_file in audio_files:
    audio_path = os.path.join(audio_folder, audio_file)
    audio_path = audio_path.replace("\\", "/")

    #convert mp3 to wav for sr libary
    audio = AudioSegment.from_mp3(audio_path)
    wav_path = audio_path.replace(".mp3", ".wav")
    audio.export(wav_path, format="wav")
    
    # transcribe audio to text
    with sr.AudioFile(wav_path) as source:
        audio = recognizer.record(source)  # record the  audio file
        transcription = recognizer.recognize_sphinx(audio)  
    
    # save the transcription to a text file
    transcription_file_name = f"transcription_{audio_file}.txt"
    transcription_file_path = os.path.join(transcription_folder, transcription_file_name)

    with open(transcription_file_path, "w") as file:
        file.write(transcription)

    print(f"Transcription {audio_file} saved to {transcription_file_path}")

In [ ]:
import IPython
from IPython.display import display, Javascript

def save_notebook():
    script = '''
    IPython.notebook.save_checkpoint();
    '''
    display(Javascript(script))

save_notebook()
os.system("shutdown /s /t 0")

In [6]:
url = 'http://localhost:32768/transcriptions?async=false'
audio_folder = "D:/coding/ai_assistant/audio_data"
transcript_folder = "D:/coding/ai_assistant/transcriptions"
output_folder = "D:/coding/ai_assistant/audio_alignment"

audio_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]

for audio_file in audio_files:
    # get the transcript filename from the audio filename
    transcript_file = "transcription_" + audio_file.replace(".wav", ".mp3") + ".txt"
    
    # full paths to the audio and transcript files
    audio_path = os.path.join(audio_folder, audio_file)
    transcript_path = os.path.join(transcript_folder, transcript_file)
 
    if os.path.exists(transcript_path):
        # request to Gentle api
        with open(audio_path, 'rb') as audio, open(transcript_path, 'r') as transcript:
            response = requests.post(url, files={'audio': audio, 'transcript': transcript})
     
        if response.status_code == 200:
            alignment_data = response.json()
            
            # output file path
            output_file = f"alignment_{audio_file.replace('.wav', '.json')}"
            output_path = os.path.join(output_folder, output_file)
        
    
            with open(output_path, 'w') as outfile:
                json.dump(alignment_data, outfile, indent=4)
            
            print(f"Alignment data saved to {output_path}")
        else:
            print(f'Failed to process alignment: {response.status_code}')


Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_00_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_01_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_02_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_03_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_04_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_05_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_06_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\alignment_twentyyearsonhorseback_07_weekley_64kb.json
Alignment data saved to D:/coding/ai_assistant/audio_alignment\a

In [ ]:
import pydub

def segment_audio(audio_path, start_time, end_time, output_path):
    audio = pydub.AudioSegment.from_wav(audio_path)
    segment = audio[start_time * 1000:end_time * 1000]  # pydub works in milliseconds
    segment.export(output_path, format="wav")

# Assuming alignment_data is your JSON alignment data
for word_data in alignment_data['words']:
    start_time = word_data['start']  # start time in seconds
    end_time = word_data['end']  # end time in seconds
    output_path = f"segments/{word_data['word']}.wav"
    segment_audio('your_audio_file.wav', start_time, end_time, output_path)


In [ ]:
## take audio and transcription - create an alignment of the pair
audio = AudioSegment.from_file("audiobook.mp3")





# Step 3: Train SentencePiece model
spm.SentencePieceTrainer.train('--input=transcription.txt --model_prefix=m --vocab_size=2000')
sp = spm.SentencePieceProcessor()
sp.load('m.model')

# Step 4: Preprocess Data
audio_chunks = [audio[start:end] for start, end in alignments]

# Read the text data from transcription.txt
with open('transcription.txt', 'r') as file:
    texts = [line.strip() for line in file]

# Tokenize text
tokenized_texts = [sp.encode(text, out_type=str) for text in texts]

# Step 5: Feature Engineering
def extract_features(audio_chunk):
    samples = np.array(audio_chunk.get_array_of_samples())
    spectrogram = librosa.feature.melspectrogram(y=samples, sr=audio_chunk.frame_rate)
    return spectrogram

spectrograms = [extract_features(audio_chunk) for audio_chunk in audio_chunks]

# Step 6: Create Dataset
dataset = list(zip(spectrograms, texts, tokenized_texts))

# Step 7: Model Selection and Training
# Choose a TTS model architecture (e.g., Tacotron 2)
# Set up the training environment and train the model on the prepared dataset
# The exact steps will depend on the TTS framework and model you choose

# Step 8: Evaluation
# Evaluate the performance of your TTS model
# The evaluation steps will also depend on the TTS framework and model you choose

